In [37]:
import numpy as np
import pandas as pd
import glob

# Read data
for filename in glob.iglob("sp500_taq/*.dta"):
    # get permnos
    df = pd.read_stata(filename)
    unique_permnos = df.permno.unique()
    print(unique_permnos)
    with open("permnos.txt", "ab") as f:
        np.savetxt(f, unique_permnos, fmt="%i")
        
# Code has generated permnos.txt file with all the permnos
# sort -u permnos.txt >> output.txt
# This unix command will only save the unique permnos

In [22]:
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm

# Leaves year + month, disregards day
def new_date(date_old):
    return str(date_old)[:6]

beta = pd.read_csv('betas.csv')
beta.drop(['n', 'RET', 'alpha', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
# Keep the last dates of each unique date
beta['new_date'] = beta['DATE'].apply(new_date)

# New Beta df
new_beta = pd.DataFrame()
# Unique dates
beta_dates = beta.new_date.unique()

for date in tqdm(beta_dates):
    date_beta = beta.loc[beta['new_date'] == date]
    permnos = date_beta.PERMNO.unique()
    for permno in permnos:
        temp = date_beta.loc[date_beta['PERMNO'] == permno].copy()
        new_beta = new_beta.append(temp.tail(1), ignore_index=True)

# Drop irrelevant columns
new_beta.drop(['new_date'], axis=1, inplace=True)

# Function to change dates of beta.csv
def convert_date(date_old):
    year = int(str(date_old)[:4]) # 2019
    month = int(str(date_old)[4:6]) # 02
    if month == 12:
        # December, so change to next year Jan
        date_new = str(year + 1) + '-01'
    else:
        # Not December
        date_new = str(year) + '-' + str(month + 1).zfill(2)
    
    return date_new

# Convert DATE column
new_beta['DATE'] = new_beta['DATE'].apply(convert_date)

new_beta.to_csv('beta_use.csv', encoding='utf-8', index=False)

KeyboardInterrupt: 

In [1]:
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm

# Same as above, but for b_umd

# Leaves year + month, disregards day
def new_date(date_old):
    return str(date_old)[:6]

beta = pd.read_csv('umd.csv')
beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml','ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
# Keep the last dates of each unique date
beta['new_date'] = beta['DATE'].apply(new_date)

# New Beta df
new_beta = pd.DataFrame()
# Unique dates
beta_dates = beta.new_date.unique()

for date in tqdm(beta_dates):
    date_beta = beta.loc[beta['new_date'] == date]
    permnos = date_beta.PERMNO.unique()
    for permno in permnos:
        temp = date_beta.loc[date_beta['PERMNO'] == permno].copy()
        new_beta = new_beta.append(temp.tail(1), ignore_index=True)

# Drop irrelevant columns
new_beta.drop(['new_date'], axis=1, inplace=True)

# Function to change dates of beta.csv
def convert_date(date_old):
    year = int(str(date_old)[:4]) # 2019
    month = int(str(date_old)[4:6]) # 02
    if month == 12:
        # December, so change to next year Jan
        date_new = str(year + 1) + '-01'
    else:
        # Not December
        date_new = str(year) + '-' + str(month + 1).zfill(2)
    
    return date_new

# Convert DATE column
new_beta['DATE'] = new_beta['DATE'].apply(convert_date)

new_beta.to_csv('umd_use.csv', encoding='utf-8', index=False)

100%|██████████| 300/300 [22:46<00:00,  4.56s/it]


In [6]:
# Same as above, but for ivol and tvol
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm

# Leaves year + month, disregards day
def new_date(date_old):
    return str(date_old)[:6]

beta = pd.read_csv('umd.csv')
beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'b_umd','R2', 'exret'], axis=1, inplace=True)
# Keep the last dates of each unique date
beta['new_date'] = beta['DATE'].apply(new_date)

# New Beta df
new_beta = pd.DataFrame()
# Unique dates
beta_dates = beta.new_date.unique()

for date in tqdm(beta_dates):
    date_beta = beta.loc[beta['new_date'] == date]
    permnos = date_beta.PERMNO.unique()
    for permno in permnos:
        temp = date_beta.loc[date_beta['PERMNO'] == permno].copy()
        new_beta = new_beta.append(temp.tail(1), ignore_index=True)

# Drop irrelevant columns
new_beta.drop(['new_date'], axis=1, inplace=True)

# Function to change dates of beta.csv
def convert_date(date_old):
    year = int(str(date_old)[:4]) # 2019
    month = int(str(date_old)[4:6]) # 02
    if month == 12:
        # December, so change to next year Jan
        date_new = str(year + 1) + '-01'
    else:
        # Not December
        date_new = str(year) + '-' + str(month + 1).zfill(2)
    
    return date_new

# Convert DATE column
new_beta['DATE'] = new_beta['DATE'].apply(convert_date)

new_beta.to_csv('vol_use.csv', encoding='utf-8', index=False)

100%|██████████| 300/300 [1:01:50<00:00, 12.37s/it]


In [23]:
# Assign portfolio number
new_beta = pd.read_csv('beta_use.csv')
mbeta = pd.DataFrame()
strategies = ['b_mkt', 'b_smb', 'b_hml']

# Assign portfolio numbers only to permnos that belong in specific date
for filename in glob.iglob("sp500_taq/*.dta"):
    df = pd.read_stata(filename)
    # Get unique dates for this file
    unique_dates = df.td.unique()
    # Isolate permnos for dates
    for date in unique_dates:
        date_df = df.loc[df['td'] == date]
        date_permnos = date_df.permno.unique()
        # Bucket betas
        date_beta = new_beta.loc[new_beta['DATE'].str.match(str(date)[:7]) & new_beta['PERMNO'].isin(date_permnos)].copy()
        # Iterate all strategies
        date_beta['b_mkt_port'] = pd.qcut(date_beta['b_mkt'], 5, labels=False, duplicates='drop')
        date_beta['b_smb_port'] = pd.qcut(date_beta['b_smb'], 5, labels=False, duplicates='drop')
        date_beta['b_hml_port'] = pd.qcut(date_beta['b_hml'], 5, labels=False, duplicates='drop')

        mbeta = mbeta.append(date_beta, ignore_index=True)

mbeta.to_csv('isolated_beta_port.csv', encoding='utf-8', index=False)

In [2]:
# Same as above but for b_umd

# Assign portfolio number
new_beta = pd.read_csv('umd_use.csv')
mbeta = pd.DataFrame()

# Assign portfolio numbers only to permnos that belong in specific date
for filename in glob.iglob("sp500_taq/*.dta"):
    df = pd.read_stata(filename)
    # Get unique dates for this file
    unique_dates = df.td.unique()
    # Isolate permnos for dates
    for date in unique_dates:
        date_df = df.loc[df['td'] == date]
        date_permnos = date_df.permno.unique()
        # Bucket betas
        date_beta = new_beta.loc[new_beta['DATE'].str.match(str(date)[:7]) & new_beta['PERMNO'].isin(date_permnos)].copy()
        # Iterate all strategies
        date_beta['b_umd_port'] = pd.qcut(date_beta['b_umd'], 5, labels=False, duplicates='drop')

        mbeta = mbeta.append(date_beta, ignore_index=True)

mbeta.to_csv('isolated_umd_port.csv', encoding='utf-8', index=False)

In [27]:
# Same as above but for ivol and tvol

# Assign portfolio number
new_beta = pd.read_csv('vol_use.csv')

mbeta = pd.DataFrame()

# Assign portfolio numbers only to permnos that belong in specific date
for filename in glob.iglob("sp500_taq/*.dta"):
    df = pd.read_stata(filename)
    # Get unique dates for this file
    unique_dates = df.td.unique()
    # Isolate permnos for dates
    for date in unique_dates:
        date_df = df.loc[df['td'] == date]
        date_permnos = date_df.permno.unique()
        # Bucket betas
        date_beta = new_beta.loc[new_beta['DATE'].str.match(str(date)[:7]) & new_beta['PERMNO'].isin(date_permnos)].copy()
        # Iterate all strategies
        date_beta['ivol_port'] = pd.qcut(date_beta['ivol'], 5, labels=False, duplicates='drop')
        date_beta['tvol_port'] = pd.qcut(date_beta['tvol'], 5, labels=False, duplicates='drop')

        mbeta = mbeta.append(date_beta, ignore_index=True)

mbeta.to_csv('isolated_vol_port.csv', encoding='utf-8', index=False)

In [3]:
# Final run

import numpy as np
import pandas as pd
import glob
import datetime
from tqdm.auto import tqdm

# Function to convert date into concise string format
# Ex: 2019-01-01:00:00:00 into 2019-01-01
def convert_date(date_oldformat):
    date_oldformat = str(date_oldformat)
    return date_oldformat[11:]

# Calculate hour/min of 1 minute before announcement
def get_minbefore(atime):
    ahour = int(str(atime)[:2])
    amin = int(str(atime)[3:5])
    atime = datetime.datetime(2000,1,1,ahour,amin) - datetime.timedelta(minutes=1)
    atime = str(atime)[11:] # atime is now the 1 minute before announcement time
    return atime

# Master dataframe with columns
masterdf = pd.DataFrame(columns=['date', 'b_mkt_pre_ls', 'b_mkt_post_ls', 'b_smb_pre_ls', 'b_smb_post_ls', 'b_hml_pre_ls', 'b_hml_post_ls'])

# Handle FOMC file
fomc = pd.read_excel('fomc_meetings.xlsx') # open fomc file
fomc = fomc[fomc['announcement'].notna()] # drop rows with no announcement time
fomc = fomc.loc[fomc['fomc'] == 1] # drop rows with fomc col set 0

fomc_dates = fomc.td.unique() # unique annoucement dates

# Open beta portfolios
beta = pd.read_csv('isolated_beta_port.csv')

strategies = ['b_mkt','b_smb','b_hml'] # strategies to iterate

# Store previous year of fomc announcement years to prevent unnecessarily opening data
prev_year = 0

for date in tqdm(fomc_dates):
    year = str(date)[:4] # ex: 2019
    year_month = str(date)[:7] # ex: 2019-01
    time = str(date)[:10] # ex: 2019-01-01
    
    # If file already open, skip
    if year != prev_year:
        filename = 'sp500_taq/sp500_fomc_' + year + '.dta' # create filename
        df = pd.read_stata(filename) # Open price data for the year / strat
    
    # Store previous year
    prev_year = year

    df['td'] = df['td'].astype('str') # convert col into string from datetime
    
    # df is the price data for this announcement day
    pdf = df.loc[df['td'].str.match(time)].copy() # price data for corresponding day
    pdf['tr'] = pdf['tr'].apply(convert_date) # clean col tr

    # Get announcement time
    atime = fomc.loc[fomc['td'] == date, 'announcement'].item()
    # Get one minute before announcement time
    atime = get_minbefore(atime)

    # Get beta for date
    date_beta = beta.loc[beta['DATE'] == year_month]

    temp = []

    for strat in strategies:
        col_name = strat + '_port'
        # Permnos of portfolio n
        long_permnos = date_beta.loc[date_beta[col_name] == 4, 'PERMNO'].values
        short_permnos = date_beta.loc[date_beta[col_name] == 0, 'PERMNO'].values
        # Isolate portfolio 0 and 4 (bottom and top quintile)
        long_df = pdf[pdf['permno'].isin(long_permnos)].copy()
        short_df = pdf[pdf['permno'].isin(short_permnos)].copy()
    
        # List of unique permnos for each portfolio
        long_permnos = long_df.permno.unique()
        short_permnos = short_df.permno.unique()

        # Store returns for each date here
        l_pre = [] # open to 1 min before announcement
        l_post = [] # 1 min before announcment to close

        s_pre = []
        s_post = []
    
        # Long 4
        for permno in long_permnos:
            permno_df = long_df.loc[long_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            l_pre.append(open_to_pre)
            l_post.append(pre_to_close)

        # Short 0
        for permno in short_permnos:
            permno_df = short_df.loc[short_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            s_pre.append(open_to_pre)
            s_post.append(pre_to_close)
        
        # Now, the returns for the announcement date are calculated
        l_pre_avg = sum(l_pre) / len(l_pre)
        l_post_avg = sum(l_post) / len(l_post)

        s_pre_avg = sum(s_pre) / len(s_pre)
        s_post_avg = sum(s_post) / len(s_post)
        
        temp.append(l_pre_avg - s_pre_avg)
        temp.append(l_post_avg - s_post_avg)

    # Append row
    masterdf = masterdf.append({'date': date, 'b_mkt_pre_ls': temp[0], 'b_mkt_post_ls': temp[1], 'b_smb_pre_ls': temp[2], 'b_smb_post_ls': temp[3], 'b_hml_pre_ls': temp[4], 'b_hml_post_ls': temp[5]}, ignore_index=True)

# Export as csv
masterdf.to_csv('announcement_ret.csv', encoding='utf-8', index=False)



  0%|          | 0/200 [00:00<?, ?it/s]loop


  0%|          | 1/200 [00:15<51:09, 15.42s/it]loop


  1%|          | 2/200 [00:18<39:07, 11.85s/it]loop


  2%|▏         | 3/200 [00:22<30:45,  9.37s/it]loop


  2%|▏         | 4/200 [00:26<24:55,  7.63s/it]loop


  2%|▎         | 5/200 [00:29<20:49,  6.41s/it]loop


KeyboardInterrupt: 

In [3]:
# LS Portfolio Return Calculation for b_umd
import numpy as np
import pandas as pd
import glob
import datetime
from tqdm.auto import tqdm

# Function to convert date into concise string format
# Ex: 2019-01-01:00:00:00 into 2019-01-01
def convert_date(date_oldformat):
    date_oldformat = str(date_oldformat)
    return date_oldformat[11:]

# Calculate hour/min of 1 minute before announcement
def get_minbefore(atime):
    ahour = int(str(atime)[:2])
    amin = int(str(atime)[3:5])
    atime = datetime.datetime(2000,1,1,ahour,amin) - datetime.timedelta(minutes=1)
    atime = str(atime)[11:] # atime is now the 1 minute before announcement time
    return atime

# Master dataframe with columns
masterdf = pd.DataFrame(columns=['date', 'b_umd_pre_ls', 'b_umd_post_ls'])

# Handle FOMC file
fomc = pd.read_excel('fomc_meetings.xlsx') # open fomc file
fomc = fomc[fomc['announcement'].notna()] # drop rows with no announcement time
fomc = fomc.loc[fomc['fomc'] == 1] # drop rows with fomc col set 0

fomc_dates = fomc.td.unique() # unique annoucement dates

# Open beta portfolios
beta = pd.read_csv('isolated_umd_port.csv')

strategies = ['b_umd'] # strategies to iterate

# Store previous year of fomc announcement years to prevent unnecessarily opening data
prev_year = 0

for date in tqdm(fomc_dates):
    year = str(date)[:4] # ex: 2019
    year_month = str(date)[:7] # ex: 2019-01
    time = str(date)[:10] # ex: 2019-01-01
    
    # If file already open, skip
    if year != prev_year:
        filename = 'sp500_taq/sp500_fomc_' + year + '.dta' # create filename
        df = pd.read_stata(filename) # Open price data for the year / strat
    
    # Store previous year
    prev_year = year

    df['td'] = df['td'].astype('str') # convert col into string from datetime
    
    # df is the price data for this announcement day
    pdf = df.loc[df['td'].str.match(time)].copy() # price data for corresponding day
    pdf['tr'] = pdf['tr'].apply(convert_date) # clean col tr

    # Get announcement time
    atime = fomc.loc[fomc['td'] == date, 'announcement'].item()
    # Get one minute before announcement time
    atime = get_minbefore(atime)

    # Get beta for date
    date_beta = beta.loc[beta['DATE'] == year_month]

    temp = []

    for strat in strategies:
        col_name = strat + '_port'
        # Permnos of portfolio n
        long_permnos = date_beta.loc[date_beta[col_name] == 4, 'PERMNO'].values
        short_permnos = date_beta.loc[date_beta[col_name] == 0, 'PERMNO'].values
        # Isolate portfolio 0 and 4 (bottom and top quintile)
        long_df = pdf[pdf['permno'].isin(long_permnos)].copy()
        short_df = pdf[pdf['permno'].isin(short_permnos)].copy()
    
        # List of unique permnos for each portfolio
        long_permnos = long_df.permno.unique()
        short_permnos = short_df.permno.unique()

        # Store returns for each date here
        l_pre = [] # open to 1 min before announcement
        l_post = [] # 1 min before announcment to close

        s_pre = []
        s_post = []
    
        # Long 4
        for permno in long_permnos:
            permno_df = long_df.loc[long_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            l_pre.append(open_to_pre)
            l_post.append(pre_to_close)

        # Short 0
        for permno in short_permnos:
            permno_df = short_df.loc[short_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            s_pre.append(open_to_pre)
            s_post.append(pre_to_close)
        
        # Now, the returns for the announcement date are calculated
        l_pre_avg = sum(l_pre) / len(l_pre)
        l_post_avg = sum(l_post) / len(l_post)

        s_pre_avg = sum(s_pre) / len(s_pre)
        s_post_avg = sum(s_post) / len(s_post)
        
        temp.append(l_pre_avg - s_pre_avg)
        temp.append(l_post_avg - s_post_avg)

    # Append row
    masterdf = masterdf.append({'date': date, 'b_umd_pre_ls': temp[0], 'b_umd_post_ls': temp[1]}, ignore_index=True)

# Export as csv
masterdf.to_csv('umd_ret.csv', encoding='utf-8', index=False)

100%|██████████| 200/200 [11:54<00:00,  3.57s/it]


In [28]:
# Calculate LS portfolio returns for ivol & tvol

import numpy as np
import pandas as pd
import glob
import datetime
from tqdm.auto import tqdm

# Function to convert date into concise string format
# Ex: 2019-01-01:00:00:00 into 2019-01-01
def convert_date(date_oldformat):
    date_oldformat = str(date_oldformat)
    return date_oldformat[11:]

# Calculate hour/min of 1 minute before announcement
def get_minbefore(atime):
    ahour = int(str(atime)[:2])
    amin = int(str(atime)[3:5])
    atime = datetime.datetime(2000,1,1,ahour,amin) - datetime.timedelta(minutes=1)
    atime = str(atime)[11:] # atime is now the 1 minute before announcement time
    return atime

# Master dataframe with columns
masterdf = pd.DataFrame(columns=['date', 'ivol_pre_ls', 'ivol_post_ls', 'tvol_pre_ls', 'tvol_post_ls'])

# Handle FOMC file
fomc = pd.read_excel('fomc_meetings.xlsx') # open fomc file
fomc = fomc[fomc['announcement'].notna()] # drop rows with no announcement time
fomc = fomc.loc[fomc['fomc'] == 1] # drop rows with fomc col set 0

fomc_dates = fomc.td.unique() # unique annoucement dates

# Open beta portfolios
beta = pd.read_csv('isolated_vol_port.csv')

strategies = ['ivol', 'tvol'] # strategies to iterate

# Store previous year of fomc announcement years to prevent unnecessarily opening data
prev_year = 0

for date in tqdm(fomc_dates):
    year = str(date)[:4] # ex: 2019
    year_month = str(date)[:7] # ex: 2019-01
    time = str(date)[:10] # ex: 2019-01-01
    
    # If file already open, skip
    if year != prev_year:
        filename = 'sp500_taq/sp500_fomc_' + year + '.dta' # create filename
        df = pd.read_stata(filename) # Open price data for the year / strat
    
    # Store previous year
    prev_year = year

    df['td'] = df['td'].astype('str') # convert col into string from datetime
    
    # df is the price data for this announcement day
    pdf = df.loc[df['td'].str.match(time)].copy() # price data for corresponding day
    pdf['tr'] = pdf['tr'].apply(convert_date) # clean col tr

    # Get announcement time
    atime = fomc.loc[fomc['td'] == date, 'announcement'].item()
    # Get one minute before announcement time
    atime = get_minbefore(atime)

    # Get beta for date
    date_beta = beta.loc[beta['DATE'] == year_month]

    temp = []

    for strat in strategies:
        col_name = strat + '_port'
        # Permnos of portfolio n
        long_permnos = date_beta.loc[date_beta[col_name] == 4, 'PERMNO'].values
        short_permnos = date_beta.loc[date_beta[col_name] == 0, 'PERMNO'].values
        # Isolate portfolio 0 and 4 (bottom and top quintile)
        long_df = pdf[pdf['permno'].isin(long_permnos)].copy()
        short_df = pdf[pdf['permno'].isin(short_permnos)].copy()
    
        # List of unique permnos for each portfolio
        long_permnos = long_df.permno.unique()
        short_permnos = short_df.permno.unique()

        # Store returns for each date here
        l_pre = [] # open to 1 min before announcement
        l_post = [] # 1 min before announcment to close

        s_pre = []
        s_post = []
    
        # Long 4
        for permno in long_permnos:
            permno_df = long_df.loc[long_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            l_pre.append(open_to_pre)
            l_post.append(pre_to_close)

        # Short 0
        for permno in short_permnos:
            permno_df = short_df.loc[short_df['permno'] == permno]
            
            # Open iprice
            open_iprice = permno_df.loc[permno_df['tr'] == '09:30:00', 'iprice'].values[0]
            # 1 min before announcement iprice
            pre_iprice = permno_df.loc[permno_df['tr'] == atime, 'iprice'].values[0]
            # End iprice
            close_iprice = permno_df.loc[permno_df['tr'] == '16:00:00', 'iprice'].values[0]
            
            # Open to pre-announcement HPR
            open_to_pre = (pre_iprice / open_iprice) - 1
            pre_to_close = (close_iprice / pre_iprice) - 1
        
            s_pre.append(open_to_pre)
            s_post.append(pre_to_close)
        
        # Now, the returns for the announcement date are calculated
        l_pre_avg = sum(l_pre) / len(l_pre)
        l_post_avg = sum(l_post) / len(l_post)

        s_pre_avg = sum(s_pre) / len(s_pre)
        s_post_avg = sum(s_post) / len(s_post)
        
        temp.append(l_pre_avg - s_pre_avg)
        temp.append(l_post_avg - s_post_avg)

    # Append row
    masterdf = masterdf.append({'date': date, 'ivol_pre_ls': temp[0], 'ivol_post_ls': temp[1], 'tvol_pre_ls': temp[2], 'tvol_post_ls': temp[3]}, ignore_index=True)

# Export as csv
masterdf.to_csv('vol_ret.csv', encoding='utf-8', index=False)

100%|██████████| 200/200 [13:40<00:00,  4.10s/it]


In [3]:
import numpy as np
import pandas as pd
import glob

# Clean output by rounding to 8 significant digits after converting to percentage from raw returns
def clean(old_ret):
    new_ret = round(old_ret * 100, 8)
    return new_ret

df = pd.read_csv('announcement_ret.csv')

df[['b_mkt_pre_ls', 'b_mkt_post_ls', 'b_smb_pre_ls', 'b_smb_post_ls', 'b_hml_pre_ls', 'b_hml_post_ls']] = df[['b_mkt_pre_ls', 'b_mkt_post_ls', 'b_smb_pre_ls', 'b_smb_post_ls', 'b_hml_pre_ls', 'b_hml_post_ls']].apply(clean)
df.to_csv('announcement_ret.csv', encoding='utf-8', index=False)



In [5]:
# Clean output for b_umd
import numpy as np
import pandas as pd
import glob

# Clean output by rounding to 8 significant digits after converting to percentage from raw returns
def clean(old_ret):
    new_ret = round(old_ret * 100, 8)
    return new_ret

df = pd.read_csv('umd_ret.csv')

df[['b_umd_pre_ls', 'b_umd_post_ls']] = df[['b_umd_pre_ls', 'b_umd_post_ls']].apply(clean)
df.to_csv('umd_ret.csv', encoding='utf-8', index=False)



In [29]:
# Clean output for ivol, tvol
import numpy as np
import pandas as pd
import glob

# Clean output by rounding to 8 significant digits after converting to percentage from raw returns
def clean(old_ret):
    new_ret = round(old_ret * 100, 8)
    return new_ret

df = pd.read_csv('vol_ret.csv')

df[['ivol_pre_ls', 'ivol_post_ls', 'tvol_pre_ls', 'tvol_post_ls']] = df[['ivol_pre_ls', 'ivol_post_ls', 'tvol_pre_ls', 'tvol_post_ls']].apply(clean)
df.to_csv('vol_ret.csv', encoding='utf-8', index=False)

